In [1]:
!pip install python-docx -q
!pip install transformers gradio pdfplumber sentencepiece -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 47.3 MB/s eta 0:00:00


In [2]:
#  Step 2: Import libraries
import gradio as gr
import pdfplumber
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [3]:
#  Step 3: Load models
tokenizer_qg = AutoTokenizer.from_pretrained("google/flan-t5-base")
model_qg = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
sentiment_pipeline = pipeline("sentiment-analysis")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [4]:
#  Step 4: Resume Text Extraction
from docx import Document

def extract_text_from_file(file):
    if file.name.endswith(".pdf"):
        import pdfplumber
        with pdfplumber.open(file.name) as pdf:
            full_text = ""
            for page in pdf.pages:
                full_text += page.extract_text() + "\n"
        return full_text.strip()

    elif file.name.endswith(".docx"):
        doc = Document(file.name)
        return "\n".join([para.text for para in doc.paragraphs])

    else:
        return "Unsupported file format. Please upload PDF or DOCX."



In [5]:
#  Step 5: Generate 5 questions using FLAN-T5
def generate_questions(resume_text, num=5):
    prompt = (
        f"Based on this resume:\n\n{resume_text[:700]}\n\n"
        f"Generate {num} diverse and unique interview questions, including technical, experience-based, and behavioral aspects. "
        "Avoid repeating similar questions. Format each question as a separate line."

    )
    inputs = tokenizer_qg(prompt, return_tensors="pt")
    outputs = model_qg.generate(**inputs, max_new_tokens=256)
    result = tokenizer_qg.decode(outputs[0], skip_special_tokens=True)

    # Extract questions containing '?'
    questions = list({q.strip("-•1234567890. ").strip() for q in result.split("\n") if "?" in q})

    # Fill or truncate to exactly num questions
    fallback = [
        "What are your key technical strengths?",
        "Can you walk me through your most impactful project?",
        "How do you handle working under pressure?",
        "What are your long-term career goals?",
        "Describe a time you worked in a team."
    ]
    while len(questions) < num:
        questions.append(fallback[len(questions)])
    return questions[:num]


In [6]:

# Step 6: Interview State
state = {
    "resume_text": "",
    "questions": [],
    "index": 0,
    "score": 0,
    "chat": []
}


In [7]:
#  Step 7: Handle resume upload
def upload_resume(file):
    text = extract_text_from_file(file)
    state["resume_text"] = text
    state["questions"] = generate_questions(text)
    state["index"] = 0
    state["score"] = 0
    state["chat"] = []

    return "Resume uploaded. Click 'Start Interview'."

In [8]:
# Step 8: Start Interview
def start_interview():
    if not state["questions"]:
        return " Upload resume first."
    q = state["questions"][state["index"]]
    state["chat"].append((" Interviewer", q))
    return q


In [9]:
#  Step 9: Answer Handling
def handle_answer(user_answer):
    idx = state["index"]
    if idx >= len(state["questions"]):
        return " Interview already completed."

    # Analyze answer
    feedback = sentiment_pipeline(user_answer)[0]
    label = feedback["label"]
    confidence = feedback["score"]

    if label == "POSITIVE":
        state["score"] += 1

    # Store chat
    state["chat"].append((" You", user_answer))
    state["chat"].append((" Interviewer", f" Feedback: Your answer sounds **{label.lower()}** (Confidence: {confidence:.2f})"))

    # Next question or end
    state["index"] += 1
    if state["index"] < len(state["questions"]):
        next_q = state["questions"][state["index"]]
        state["chat"].append((" Interviewer", next_q))
        return next_q
    else:
        final = f" Interview completed. Score: {state['score']} / {len(state['questions'])}."
        if state["score"] >= 4:
            final += "\n Great job! Confident and clear answers."
        elif state["score"] >= 2:
            final += "\n Decent effort. Try adding more details and structure."
        else:
            final += "\n Needs improvement. Try to communicate more clearly."
        state["chat"].append((" Interviewer", final))
        return final


In [10]:
#  Step 10: Gradio UI
with gr.Blocks() as demo:
    gr.Markdown(" Resume Based QNA Chatbot")

    with gr.Row():
        file_upload = gr.File(label=" Upload Resume (PDF or DOCX)", file_types=[".pdf", ".docx"])
        upload_btn = gr.Button("Analyze Resume")
        output_msg = gr.Textbox(label="Status", lines=2)

    start_btn = gr.Button(" Start Interview")
    question_box = gr.Textbox(label="Interview Question", interactive=False)
    answer_box = gr.Textbox(label="Your Answer")
    submit_btn = gr.Button("Submit Answer")
    chatbox = gr.Chatbot(label="Chat History")

    upload_btn.click(upload_resume, inputs=file_upload, outputs=output_msg)
    start_btn.click(start_interview, outputs=question_box)
    submit_btn.click(handle_answer, inputs=answer_box, outputs=question_box)
    submit_btn.click(lambda: state["chat"], outputs=chatbox)

demo.launch(

)

/tmp/ipython-input-10-823252125.py:14: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot(label="Chat History")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://34de256a6b76e000de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
